## Подготовка данных

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle

import numpy as np

from tqdm import tqdm

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Задаём фиксированное значение гиперпараметра random_state
RANDOM_STATE = 123

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


Исходный датасет состоит из 14 столбцов и 10000 строк.  
Наблюдаются пропуски в столбце `Tenure` - количество лет, которых человек является клиентом банка.  
Следующие столбцы содержат признаки: `CreditScore`, `Geography`, `Gender`, `Age`, `Tenure`, `Balance`, `NumOfProducts`, `HasCrCard`, `IsActiveMember`, `EstimatedSalary`.  
Целевой признак: `Exited` - факт ухода клиента.

In [7]:
# Переименовываем столбцы
data.columns = data.columns.str.lower()
data.columns = data.rename({'rownumber': 'row_number', 'customerid': 'customer_id', 'creditscore': 'credit_score', \
                            'numofproducts': 'num_of_products', 'hascrcard': 'has_credit_card', \
                            'isactivemember': 'is_active_member', 'estimatedsalary': 'estimated_salary'}, axis=1).columns

In [8]:
# Пропущенные значения в столбце 'tenure' запольним медианным значением
data['tenure'] = data['tenure'].fillna(data['tenure'].median())

Сохраним признаки в переменной `features` и целевой признак в переменной `train`.

In [9]:
features = data.drop(['row_number', 'customer_id', 'surname', 'exited'], axis=1)
target = data['exited']

Разделим выборки с признаками и целевым признаком на выборки обучающие и валидационные.

In [10]:
features_train, features_valid, \
target_train, target_valid = train_test_split(features, target, train_size=.6, \
                                              random_state=RANDOM_STATE, \
                                              stratify=target)
features_valid, features_test, \
target_valid, target_test = train_test_split(features_valid, target_valid, test_size=.5, \
                                             random_state=RANDOM_STATE, \
                                             stratify=target_valid)
print(f"Количество строк в обучающей выборке по классам: {np.bincount(target_train)}")
print(f"Количество строк в валидационной выборке по классам: {np.bincount(target_valid)}")
print(f"Количество строк в тестовой выборке по классам: {np.bincount(target_test)}")

Количество строк в обучающей выборке по классам: [4778 1222]
Количество строк в валидационной выборке по классам: [1592  408]
Количество строк в тестовой выборке по классам: [1593  407]


Техникой OHE(прямое кодирование) категориальные признаки в столбцах `geography`, `gender` переведём в численные.

In [11]:
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
encoder_ohe.fit(features_train[['geography', 'gender']])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [12]:
features_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_train[['geography', 'gender']])
features_train = features_train.drop(['geography', 'gender'], axis=1)

In [13]:
features_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_valid[['geography', 'gender']])
features_valid = features_valid.drop(['geography', 'gender'], axis=1)

In [14]:
features_test[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_test[['geography', 'gender']])
features_test = features_test.drop(['geography', 'gender'], axis=1)

Далее стандартизируем данные в трёх выборках с признаками.

In [15]:
scaler = StandardScaler()
columns = ['credit_score', 'age', \
           'tenure', 'balance', \
           'num_of_products', \
           'estimated_salary']
scaler.fit(features_train[columns])

StandardScaler()

In [16]:
features_train[columns] = scaler.transform(features_train[columns])

In [17]:
features_valid[columns] = scaler.transform(features_valid[columns])

In [18]:
features_test[columns] = scaler.transform(features_test[columns])

**Вывод:** пропущенные значения заменены медианным; выделены выборки, содержащие признаки и целевой признок; выборки разделены на обучающую, валидационную и тестовую; техникой OHE категориальные признаки переведены в численные; произведено масштабирование данных.

## Исследование задачи

In [19]:
data.groupby('exited')['exited'].count()

exited
0    7963
1    2037
Name: exited, dtype: int64

Соотношение ушедших клиентов к оставшимся примерно 1/4, либо 20 и 80 в процентном соотношении.  
Создадим функции, которые обучают модели, после чего обучим модели.

In [20]:
# Модель классификации деревом решений
def model_dt_classifier(features, target, \
                        depth_range=range(1, 31), \
                        class_weight=None, \
                        random_state=RANDOM_STATE):
    f1_valid_best = 0
    for depth in tqdm(depth_range):
        model_dt= DecisionTreeClassifier(max_depth=depth, random_state=random_state, class_weight=class_weight)
        model_dt.fit(features, target)
        predictions_valid = model_dt.predict(features_valid)
        f1_valid = f1_score(target_valid, predictions_valid)
        probabilities_one_valid = model_dt.predict_proba(features_valid)[:, 1]
        if f1_valid > f1_valid_best:
            f1_valid_best = f1_valid
            best_max_depth = depth
            best_model_dt = model_dt
            auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    print(f'Высший показатель f1 меры на валидационной выборке равен {f1_valid_best} при глубине дерева - {best_max_depth}.')
    print(f'Значение roc-auc равно {auc_roc}.')
    return(best_model_dt)

In [21]:
# Модель классификации случайный лес
def model_rf_classifier(features, target, \
                        estimators_range=range(1, 51), \
                        depth_range=range(1, 31), \
                        class_weight=None, \
                        random_state=RANDOM_STATE):
    f1_valid_best = 0
    for estimators in tqdm(estimators_range):
        for depth in depth_range:
            model_rf = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=random_state)
            model_rf.fit(features, target)
            predictions_valid = model_rf.predict(features_valid)
            f1_valid = f1_score(target_valid, predictions_valid)
            probabilities_one_valid = model_rf.predict_proba(features_valid)[:, 1]
            if f1_valid > f1_valid_best:
                f1_valid_best = f1_valid
                best_max_depth = depth
                best_model_rf = model_rf
                auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
                best_n_estimators = estimators
    print(f'Высший показатель f1 меры на валидационной выборке равен {f1_valid_best} при глубине дерева - {best_max_depth} и количестве деревьев - {best_n_estimators}.')
    print(f'Значение roc-auc равно {auc_roc}.')
    return(best_model_rf)

In [22]:
# Модель логистической регрессии
def model_lr(features, target, \
             solver='lbfgs', \
             class_weight=None, \
             random_state=RANDOM_STATE):
    model_lr = LogisticRegression(solver=solver, class_weight=class_weight, random_state=random_state)
    model_lr.fit(features, target)
    predictions_valid = model_lr.predict(features_valid)
    f1_valid = f1_score(target_valid, predictions_valid)
    probabilities_one_valid = model_lr.predict_proba(features_valid)[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    print(f'Показатель f1 меры на валидационной выборке равен {f1_valid}.')
    print(f'Значение roc-auc равно {auc_roc}.')
    return(model_lr)

In [23]:
%%time
# Модель классификации деревом решений без учёта дисбаланса
model_dt_unbalanced = model_dt_classifier(features_train, target_train)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 56.51it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5664739884393063 при глубине дерева - 7.
Значение roc-auc равно 0.8198360060104445.
CPU times: total: 562 ms
Wall time: 534 ms


In [24]:
%%time
# Модель классификации случайный лес без учёта дисбаланса
model_rf_unbalanced = model_rf_classifier(features_train, target_train, \
                                          estimators_range=range(25, 31), \
                                          depth_range=range(18, 23))

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.39it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5748502994011976 при глубине дерева - 20 и количестве деревьев - 29.
Значение roc-auc равно 0.8358928527441126.
CPU times: total: 4.33 s
Wall time: 4.33 s


In [25]:
# Модель логистической регрессии без учёта дисбаланса
model_lr_unbalanced = model_lr(features_train, target_train, \
                               solver='liblinear')

Показатель f1 меры на валидационной выборке равен 0.29840142095914746.
Значение roc-auc равно 0.731508338259927.


При обучении моделей без учёта дисбаланса классов наибольшие значения f1 меры и auc-roc у модели случайного леса. На втором месте модель дерева решений, затем модель логистической регрессии.

## Борьба с дисбалансом

Напишем функции:  
1) уменьшающую выборку с целевым признаком 0;  
2) увеличивающую выборку с целевым признаком 1.

In [26]:
def downsample(fraction, features=features_train, target=target_train, random_state=RANDOM_STATE):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=random_state)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=random_state)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=random_state)
    return features_downsampled, target_downsampled

In [27]:
def upsample(repeat, features=features_train, target=target_train, random_state=RANDOM_STATE):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=random_state)
    return features_upsampled, target_upsampled

Отношение целевых признаков 1/4, поэтому рассмотрим несколько вариантов балансирования классов:  
- увеличение выборки с целевым признаком 1 в четыре раза;
- уменьшение выборки с целевым признаком 0 в четыре раза;
- увеличение выборки с целевым признаком 1 в два раза и уменьшение выборки с целевым признаком 0 в два раза.

In [28]:
# Первый вариант (увеличение выборки с ушедшими клиентами в четыре раза)
features_train_balanced, target_train_balanced = upsample(4)
target_train_balanced.value_counts()

1    4888
0    4778
Name: exited, dtype: int64

In [29]:
%%time
# Модель классификации деревом решений
model_dt_balanced_first = model_dt_classifier(features_train_balanced, target_train_balanced)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 44.84it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5577981651376147 при глубине дерева - 6.
Значение roc-auc равно 0.8253414745295103.
CPU times: total: 672 ms
Wall time: 671 ms


In [30]:
%%time
# Модель классификации случайный лес
model_rf_balanced_first = model_rf_classifier(features_train_balanced, target_train_balanced, \
                                            estimators_range=range(33, 37), \
                                            depth_range=range(13, 17))

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.03it/s]

Высший показатель f1 меры на валидационной выборке равен 0.6002522068095838 при глубине дерева - 15 и количестве деревьев - 35.
Значение roc-auc равно 0.8409287552960883.
CPU times: total: 3.92 s
Wall time: 3.9 s


In [31]:
# Модель логистической регрессии
model_lr_balanced_first = model_lr(features_train_balanced, target_train_balanced, \
                                 solver='liblinear')

Показатель f1 меры на валидационной выборке равен 0.4575389948006932.
Значение roc-auc равно 0.7344011725293133.


В первом варианте наивысшее качество у модели случайный лес. 

Значение f1 меры относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.  

Значение roc-auc относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений увеличилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.

In [32]:
# Второй вариант (уменьшение выборки с оставшимися клиентами в четыре раза)
features_train_balanced, target_train_balanced = downsample(.25)
target_train_balanced.value_counts()

1    1222
0    1194
Name: exited, dtype: int64

In [33]:
%%time
# Модель классификации деревом решений
model_dt_balanced_second = model_dt_classifier(features_train_balanced, target_train_balanced)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 98.01it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5457979225684608 при глубине дерева - 6.
Значение roc-auc равно 0.8125269423095871.
CPU times: total: 312 ms
Wall time: 309 ms


In [34]:
%%time
# Модель классификации случайный лес
model_rf_balanced_second = model_rf_classifier(features_train_balanced, target_train_balanced, \
                                            estimators_range=range(10, 15), \
                                            depth_range=range(1, 10))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.04it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5749761222540593 при глубине дерева - 8 и количестве деревьев - 13.
Значение roc-auc равно 0.8335142316977042.
CPU times: total: 1.23 s
Wall time: 1.24 s


In [35]:
# Модель логистической регрессии
model_lr_balanced_second = model_lr(features_train_balanced, target_train_balanced, \
                                 solver='lbfgs')

Показатель f1 меры на валидационной выборке равен 0.457482993197279.
Значение roc-auc равно 0.7329832372647552.


Во втором варианте наивысшее качество у модели случайный лес. 

Значение f1 меры относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.  

Значение roc-auc относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес уменьшилось;
- логистической регрессии увеличилось.

In [36]:
# Третий вариант (уменьшение выборки с оставшимися клиентами в два раза и увеличение выборки ушедших клиентов в два раза)
features_train_balanced, target_train_balanced = downsample(.5)
features_train_balanced, target_train_balanced = upsample(2, features_train_balanced, target_train_balanced)
target_train_balanced.value_counts()

1    2444
0    2389
Name: exited, dtype: int64

In [37]:
%%time
# Модель классификации деревом решений
model_dt_balanced_third = model_dt_classifier(features_train_balanced, target_train_balanced)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 61.76it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5581874356333676 при глубине дерева - 6.
Значение roc-auc равно 0.8189653845206424.
CPU times: total: 1.69 s
Wall time: 488 ms


In [38]:
%%time
# Модель классификации случайный лес
model_rf_balanced_third = model_rf_classifier(features_train_balanced, target_train_balanced, \
                                            estimators_range=range(21, 26), \
                                            depth_range=range(13, 17))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.43it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5894962486602358 при глубине дерева - 15 и количестве деревьев - 24.
Значение roc-auc равно 0.8381028919105331.
CPU times: total: 2.06 s
Wall time: 2.06 s


In [39]:
# Модель логистической регрессии
model_lr_balanced_third = model_lr(features_train_balanced, target_train_balanced, \
                                 solver='lbfgs')

Показатель f1 меры на валидационной выборке равен 0.457092819614711.
Значение roc-auc равно 0.7341363681150852.


В третьем варианте наивысшее качество у модели случайный лес. 

Значение f1 меры относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.  

Значение roc-auc относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.

**Гиперпараметр weight_class.**  
Проверим вариант с изменением гиперпараметра weight_class, при этом в качестве обучающей выборки возьмём несбалансированную по классам.

In [40]:
%%time
# Модель классификации деревом решений с гиперпараметром class_weight='balanced'
model_dt_balanced_fourth = model_dt_classifier(features_train, target_train, \
                                               class_weight='balanced')

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 49.88it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5591200733272228 при глубине дерева - 6.
Значение roc-auc равно 0.8277531961276973.
CPU times: total: 2.02 s
Wall time: 603 ms


In [41]:
%%time
# Модель классификации случайный лес c гиперпараметром class_weight='balanced'
model_rf_balanced_fourth = model_rf_classifier(features_train, target_train, \
                                               estimators_range=range(31, 35), \
                                               depth_range=range(12, 16), \
                                               class_weight='balanced')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.63it/s]

Высший показатель f1 меры на валидационной выборке равен 0.5757575757575757 при глубине дерева - 14 и количестве деревьев - 33.
Значение roc-auc равно 0.8396632365257661.
CPU times: total: 2.47 s
Wall time: 2.46 s


In [42]:
# Модель логистической регрессии с гиперпараметром class_weight='balanced'
model_lr_balanced_fourth = model_lr(features_train, target_train, \
                                    solver='liblinear', \
                                    class_weight='balanced')

Показатель f1 меры на валидационной выборке равен 0.46140350877192987.
Значение roc-auc равно 0.7343519065917825.


В четвёртом варианте наивысшее качество у модели случайный лес. 

Значение f1 меры относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений уменьшилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.  

Значение roc-auc относительно варианта без учёта дисбаланса у модели:
- классификации деревом решений увеличилось;
- случайный лес увеличилось;
- логистической регрессии увеличилось.

**Вывод:** проверив модели по четырём вариантам баланса классов, выделим модель с лучшими метриками качества. Это модель случайного леса в первом варианте (увеличение выборки ушедших клиентов в 4 раза).

## Тестирование модели

Проверим модель на тестовом наборе признаков.

In [43]:
predictions_test = model_rf_balanced_first.predict(features_test)
f1_test = f1_score(target_test, predictions_test)
probabilities_one_test = model_rf_balanced_first.predict_proba(features_test)[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(f'На тестовой выборке значение f1 меры равно {f1_test}, значение roc-auc - {auc_roc}.')

На тестовой выборке значение f1 меры равно 0.6357947434292867, значение roc-auc - 0.8569987553038401.


In [44]:
dummy_clf = DummyClassifier()
dummy_clf.fit(features_train, target_train)
dummy_probabilities_one_test = dummy_clf.predict_proba(features_test)[:, 1]
auc_roc = roc_auc_score(target_test, dummy_probabilities_one_test)
print(f'значение roc-auc у случайной модели - {auc_roc}.')

значение roc-auc у случайной модели - 0.5.


## **Общий вывод:**

**Подготовка данных.**  
Суть подготовки заключается в приведении данных к формату для дальнейшего обучения моделей. В процессе подготовки исходный датасет был разделён на выборки с признаками и целевыми признаками, затем на тренировочную, валидационную и тестовую выборки. Техникой OHE категориальные признаки переведены в численные. Произведено масштабирование данных.  
**Исследование задачи.**  
Изучен дисбаланс классов. отношение доли ушедших клиентов к оставшимся составляет примерно 1/4. Обучены модели без учёта дисбаланса.  
**Борьба с дисбалансом.**  
Рассмотрено несколько вариантов баланса классов путём увеличения и уменьшения выборок с разными целевыми признаками.  
Обучены модели по сбалансированным выборкам, найдена модель с высшими значениями f1 меры и roc-auc - это модель случайного леса при увеличении выборки ушедших клиентов в четыре раза.  
**Тестирование модели.**  
На тестовой выборке удалось достичь значение f1 меры 0.6357947434292867, что выше необходимого. Значение площади под кривой ошибок равно 0.8569987553038401. Значение roc-auc у случайной модели - 0.5, следовательно качество модели случайного леса выше.